In [2]:
!pip3 install tensorflow 
!pip3 install yahoo_fin 
!pip3 install requests_html

     |████████████████████████████████| 153kB 9.9MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=79549838b63a153919720982fbc0615fceb34ad1e9d83f8bf1eff114ef6a4755
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24133 sha256=0768cfaee889695127f6674fd7c5a7f367f76030bb391907d91d2dc198ad2840
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
Successfully built fake-useragent parse
ERROR: pyppeteer 0.2.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from yahoo_fin import stock_info as si
from collections import deque
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime, timedelta
import csv
import pandas_datareader.data as web
import pandas_datareader as pdr
from pandas_datareader import data, wb
import time
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like #For solving import pandas_datareader issue
import os
import random
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import urlopen

In [ ]:

wiki_url = 'https://en.wikipedia.org/wiki/NIFTY_50'    
html1=urlopen(wiki_url)
soup = BeautifulSoup(html1, 'html.parser')
nse50data1=""

table = soup.find_all('table', class_="wikitable")[0] 

for record in table.findAll('tr'):
    nse50data=""
    for data in record.findAll('td'):
        nse50data=nse50data+","+data.text
    nse50data1=nse50data1+"\n"+nse50data[1:]

print(nse50data1)
f= open("nse50.txt","w+")

f.write(nse50data1)
f.close()
head = []
for th in table.find("tr").find_all("th"):
        head.append(th.text.strip())
rows_l = []
for tr in table.find_all("tr")[1:]:
        cell_l = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
           
            ths = tr.find_all("th")
            for th in ths:
                cell_l.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cell_l.append(td.text.strip())
        rows_l.append(cell_l)
d=pd.DataFrame(rows_l, columns=head).to_csv(f"table.csv")



Adani Ports,ADANIPORTS.NS,Infrastructure

Asian Paints,ASIANPAINT.NS,Consumer Goods

Axis Bank,AXISBANK.NS,Banking

Bajaj Auto,BAJAJ-AUTO.NS,Automobile

Bajaj Finance,BAJFINANCE.NS,Financial Services

Bajaj Finserv,BAJAJFINSV.NS,Financial Services

Bharti Airtel,BHARTIARTL.NS,Telecommunication

Bharat Petroleum,BPCL.NS,Energy - Oil & Gas

Britannia Industries,BRITANNIA.NS,Consumer Goods

Cipla,CIPLA.NS,Pharmaceuticals

Coal India,COALINDIA.NS,Energy & Mining

Divi's Laboratories,DIVISLAB.NS,Pharmaceuticals

Dr. Reddy's Laboratories,DRREDDY.NS,Pharmaceuticals

Eicher Motors,EICHERMOT.NS,Automobile

GAIL,GAIL.NS,Energy - Oil & Gas

Grasim Industries,GRASIM.NS,Cement

HCL Technologies,HCLTECH.NS,Information Technology

HDFC,HDFC.NS,Financial Services

HDFC Bank,HDFCBANK.NS,Banking

HDFC Life,HDFCLIFE.NS,Insurance

Hero MotoCorp,HEROMOTOCO.NS,Automobile

Hindalco Industries,HINDALCO.NS,Metals

Hindustan Unilever,HINDUNILVR.NS,Consumer Goods

ICICI Bank,ICICIBANK.NS,Banking

IndusInd Bank

In [ ]:
# Import list of stock names from NSE website
with requests.Session() as sen:
    down= sen.get('https://www1.nseindia.com/products/content/sec_bhavdata_full.csv')
    decoded= down.content.decode('utf-8')
    csv_read = csv.reader(decoded.splitlines(), delimiter=',')
    df = pd.DataFrame(list(csv_read))

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,SYMBOL,SERIES,DATE1,PREV_CLOSE,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,LAST_PRICE,CLOSE_PRICE,AVG_PRICE,TTL_TRD_QNTY,TURNOVER_LACS,NO_OF_TRADES,DELIV_QTY,DELIV_PER
1,20MICRONS,EQ,29-Sep-2020,29.10,29.10,29.70,28.35,29.15,28.60,28.83,17085,4.93,289,9484,55.51
2,21STCENMGM,EQ,29-Sep-2020,10.25,10.25,10.35,10.05,10.05,10.05,10.12,1050,0.11,16,1050,100.00
3,3IINFOTECH,EQ,29-Sep-2020,3.00,3.10,3.15,3.05,3.15,3.15,3.12,2098201,65.53,1083,1216528,57.98
4,3MINDIA,EQ,29-Sep-2020,18361.85,18499.95,18550.00,18100.00,18384.00,18399.70,18344.78,3576,656.01,1657,1145,32.02


In [4]:
np.random.seed(314)
tf.random.set_seed(314)
random.seed(314)

In [1]:

print("(CHOOSE THE TICKER FROM DATAFRAME MENTIONED ABOVE)")
print("\nEnter the ticker")
stock_in=input()
def load_data(tick=stock_in, n_step=50, scale=True, shuffle=True, lookup_step=1, 
                test_size=0.2, feat_columns=['adjclose', 'volume', 'open', 'high', 'low']):
    """
    Variable description
    Loads data from Yahoo Finance source and scaling, shuffling, normalizing and splitting.
    Parameters:
        tick (str/pd.DataFrame): the ticker you want to load
        n_step(int): the historical sequence length (i.e window size) used to predict, default is 50
        scale (bool): whether to scale prices from 0 to 1, default is True
        shuffle (bool): whether to shuffle the data, default is True
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        test_size (float): ratio for test data, default is 0.2 (20% testing data)
        feat_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(tick, str):
        # load it from yahoo_fin library
        df = si.get_data(tick)
    elif isinstance(tick, pd.DataFrame):
        # already loaded, use it directly
        df = tick
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    res = {}
    # we will also return the original dataframe itself
    res['df'] = df.copy()
    # make sure that the passed feat_columns exist in the dataframe
    for col in feat_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
    if scale:
        scaler_column = {}
        # scale the data (prices) from 0 to 1
        for column in feat_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            scaler_column[column] = scaler
        # add the MinMaxScaler instances to the res returned
        res["scaler_column"] = scaler_column
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
  
    seq_last = np.array(df[feat_columns].tail(lookup_step))
  
    df.dropna(inplace=True)
    data_seq = []
    sequences = deque(maxlen=n_step)
    for entry, target in zip(df[feature_columns].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_step:
            data_seq.append([np.array(sequences), target])
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence

    # this seq_last will be used to predict future stock prices not available in the dataset
    seq_last = list(sequences) + list(seq_last)
    seq_last = np.array(seq_last)
    
    res['seq_last'] = seq_last
    # construct the X's and y's
    X, y = [], []
    for seq, target in data_seq:
        X.append(seq)
        y.append(target)
    
    X = np.array(X)
    y = np.array(y)
    # reshape X to fit the neural network
    X = X.reshape((X.shape[0], X.shape[2], X.shape[1]))
    # split the dataset
    res["X_train"], res["X_test"], res["y_train"], res["y_test"] = train_test_split(X, y, 
   
    return res

(CHOOSE THE TICKER FROM DATAFRAME MENTIONED ABOVE)

Enter the ticker
GAIL.NS
